In [106]:
import requests
from IPython.display import display, HTML

class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, data):
        if not self.head:
            self.head = Node(data)
            return
        current = self.head
        while current.next:
            current = current.next
        current.next = Node(data)

    def display(self):
        current = self.head
        while current:
            display(HTML(f"<p>{current.data}</p>"))
            current = current.next

def fetch_price(platform, symbol):
    """
    Fetches the price of a cryptocurrency from the specified platform.

    Args:
    - platform (str): The platform to fetch the price from.
    - symbol (str): The symbol of the cryptocurrency.

    Returns:
    - float: The price of the cryptocurrency.
    """
    urls = {
        'Coinbase': f'https://api.coinbase.com/v2/prices/{symbol.upper()}-USD/spot',
        'CryptoCompare': f'https://min-api.cryptocompare.com/data/price?fsym={symbol.upper()}&tsyms=USD',
        'Kraken': f'https://api.kraken.com/0/public/Ticker?pair={symbol.upper()}USD',
        'Bitstamp': f'https://www.bitstamp.net/api/v2/ticker/{symbol.lower()}usd/'
        # Add more platforms here if needed
    }

    if platform not in urls:
        print(f"Error: Unsupported platform '{platform}'")
        return 0.0

    url = urls[platform]
    try:
        print(f"Fetching data from {platform} API with URL: {url}")
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()

        if platform == 'Coinbase':
            return float(data['data']['amount'])
        elif platform == 'CryptoCompare':
            return float(data['USD'])
        elif platform == 'Kraken':
            return float(data['result'][list(data['result'].keys())[0]]['c'][0])
        elif platform == 'Bitstamp':
            return float(data['last'])
        # Add more platform-specific parsing logic here if needed
    except requests.RequestException as e:
        print(f"Error: Could not fetch price from {platform} API - {e}")
        return 0.0

def calculate_profit(price_buy, price_sell):
    """
    Calculates the profit based on the price difference between buying and selling.

    Args:
    - price_buy (float): The price at which the cryptocurrency is bought.
    - price_sell (float): The price at which the cryptocurrency is sold.

    Returns:
    - float: The profit based on the price difference.
    """
    return price_sell - price_buy

def fetch_prices_and_calculate_profits():
    """
    Fetches cryptocurrency prices from multiple platforms for various currencies,
    and calculates the profits based on price differences.

    Returns:
    - dict: A dictionary containing the fetched prices and calculated profits.
    """
    platforms = ['Coinbase', 'CryptoCompare', 'Kraken', 'Bitstamp']
    symbols = ['BTC', 'ETH']  # Ensure no duplicate currencies
    
    prices = {platform: {} for platform in platforms}
    for platform in platforms:
        for symbol in symbols:
            prices[platform][symbol] = fetch_price(platform, symbol)

    print(f"Prices fetched: {prices}")

    profits = {symbol: [] for symbol in symbols}
    for symbol in symbols:
        for platform_buy in platforms:
            for platform_sell in platforms:
                if platform_buy != platform_sell:
                    profit = calculate_profit(prices[platform_buy][symbol], prices[platform_sell][symbol])
                    profits[symbol].append({
                        'buy_from': platform_buy,
                        'sell_to': platform_sell,
                        'profit': round(profit, 3)
                    })

    print(f"Profits calculated: {profits}")

    return {
        'prices': prices,
        'profits': profits
    }

def display_best_trading_opportunities(data, symbols):
    """
    Displays the best trading opportunities based on the fetched prices and calculated profits.

    Args:
    - data (dict): A dictionary containing the fetched prices and calculated profits.
    - symbols (list): A list of symbols for cryptocurrencies.
    """
    profits = data['profits']

    best_trades = {symbol: max(profits[symbol], key=lambda x: x['profit']) for symbol in profits}

    html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Best Trading Opportunities (Profit per 1 {symbols[0]})</title>
        <style>
            table, th, td {{
                border: 1px solid black;
                border-collapse: collapse;
                padding: 8px;
            }}
        </style>
    </head>
    <body>
        <h1>Best Trading Opportunities (Profit per 1 {symbols[0]})</h1>
        <table>
            <tr>
                <th>Cryptocurrency</th>
                <th>Buy From</th>
                <th>Sell To</th>
                <th>Profit</th>
            </tr>
    """
    for symbol in best_trades:
        html_content += f"""
            <tr>
                <td>{symbol}</td>
                <td>{best_trades[symbol]['buy_from']}</td>
                <td>{best_trades[symbol]['sell_to']}</td>
                <td>{best_trades[symbol]['profit']:.10f}</td>  # Adjust the precision here
            </tr>
        """
    html_content += """
        </table>
    </body>
    </html>
    """

    display(HTML(html_content))

# Fetch prices and calculate profits
data = fetch_prices_and_calculate_profits()

# Display best trading opportunities
display_best_trading_opportunities(data, ['BTC', 'ETH'])


Fetching data from Coinbase API with URL: https://api.coinbase.com/v2/prices/BTC-USD/spot
Fetching data from Coinbase API with URL: https://api.coinbase.com/v2/prices/ETH-USD/spot
Fetching data from CryptoCompare API with URL: https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD
Fetching data from CryptoCompare API with URL: https://min-api.cryptocompare.com/data/price?fsym=ETH&tsyms=USD
Fetching data from Kraken API with URL: https://api.kraken.com/0/public/Ticker?pair=BTCUSD
Fetching data from Kraken API with URL: https://api.kraken.com/0/public/Ticker?pair=ETHUSD
Fetching data from Bitstamp API with URL: https://www.bitstamp.net/api/v2/ticker/btcusd/
Fetching data from Bitstamp API with URL: https://www.bitstamp.net/api/v2/ticker/ethusd/
Prices fetched: {'Coinbase': {'BTC': 66675.955, 'ETH': 3094.045}, 'CryptoCompare': {'BTC': 66674.5, 'ETH': 3094.2}, 'Kraken': {'BTC': 66652.6, 'ETH': 3093.75}, 'Bitstamp': {'BTC': 66662.0, 'ETH': 3094.0}}
Profits calculated: {'BTC': [{'bu

Cryptocurrency,Buy From,Sell To,Profit
BTC,Kraken,Coinbase,23.3550000000
ETH,Kraken,CryptoCompare,0.4500000000


In [109]:
import unittest
from unittest.mock import patch, MagicMock
from IPython.core.display import HTML
import requests


class LinkedList:
    class Node:
        def __init__(self, data):
            self.data = data
            self.next = None

    def __init__(self):
        self.head = None

    def append(self, data):
        if not self.head:
            self.head = self.Node(data)
        else:
            current = self.head
            while current.next:
                current = current.next
            current.next = self.Node(data)

    def display(self):
        current = self.head
        while current:
            display(HTML(f"<p>{current.data}</p>"))  # Modified formatting
            current = current.next


class TestLinkedList(unittest.TestCase):
    def test_linked_list_display(self):
        # Test linked list display method
        linked_list = LinkedList()
        linked_list.append(1)
        linked_list.append(2)
        linked_list.append(3)

        expected_calls = [
            MagicMock(spec=HTML, id='1'),
            MagicMock(spec=HTML, id='2'),
            MagicMock(spec=HTML, id='3')
        ]

        with patch('IPython.core.display.display', side_effect=expected_calls) as mock_display:
            linked_list.display()

        # Prepare the expected HTML content
        expected_html_content = [f"<p>{i}</p>" for i in range(1, 4)]

        # Check if the calls match the expected HTML content
        for call, expected_html in zip(mock_display.call_args_list, expected_html_content):
            actual_html = call[0][0]._repr_html_()
            self.assertEqual(actual_html, expected_html)

        # Print actual calls for debugging
        print("Actual calls:")
        for call in mock_display.call_args_list:
            print(call)


class TestFetchPricesAndCalculateProfits(unittest.TestCase):
    @patch('requests.get')
    def test_fetch_price(self, mock_get):
        # Test fetch_price function
        mock_response_coinbase = MagicMock()
        mock_response_coinbase.json.return_value = {'data': {'amount': '100'}}
        mock_response_cryptocompare = MagicMock()
        mock_response_cryptocompare.json.return_value = {'USD': 110}
        mock_response_kraken = MagicMock()
        mock_response_kraken.json.return_value = {'result': {'XXBTZUSD': {'c': ['110']}}}
        mock_response_bitstamp = MagicMock()
        mock_response_bitstamp.json.return_value = {'last': '120'}

        mock_get.side_effect = [mock_response_coinbase, mock_response_cryptocompare, mock_response_kraken,
                                mock_response_bitstamp]

        self.assertEqual(fetch_price('Coinbase', 'BTC'), 100)
        self.assertEqual(fetch_price('CryptoCompare', 'BTC'), 110)
        self.assertEqual(fetch_price('Kraken', 'BTC'), 110)
        self.assertEqual(fetch_price('Bitstamp', 'BTC'), 120)

    def test_calculate_profit(self):
        # Test calculate_profit function
        self.assertEqual(calculate_profit(100, 120), 20)
        self.assertEqual(calculate_profit(100, 100), 0)
        self.assertEqual(calculate_profit(120, 100), -20)

    def test_fetch_prices_and_calculate_profits(self):
        # Test fetch_prices_and_calculate_profits function
        data = fetch_prices_and_calculate_profits()

        self.assertIsInstance(data, dict)
        self.assertIn('prices', data)
        self.assertIn('profits', data)

        self.assertIsInstance(data['prices'], dict)
        self.assertIsInstance(data['profits'], dict)

        self.assertIn('Coinbase', data['prices'])
        self.assertIn('BTC', data['prices']['Coinbase'])

        self.assertIn('BTC', data['profits'])
        self.assertIsInstance(data['profits']['BTC'], list)
        self.assertTrue(len(data['profits']['BTC']) > 0)


# Run the tests directly in the notebook cell
unittest.main(argv=[''], verbosity=2, exit=False)


test_calculate_profit (__main__.TestCalculateProfit) ... ok
test_display_best_trading_opportunities (__main__.TestDisplayBestTradingOpportunities) ... 

Cryptocurrency,Buy From,Sell To,Profit
BTC,Coinbase,Bitstamp,10.0000000000
ETH,Coinbase,Kraken,20.0000000000


ok
test_fetch_price (__main__.TestFetchPrice) ... ok
test_calculate_profit (__main__.TestFetchPricesAndCalculateProfits) ... ok
test_fetch_price (__main__.TestFetchPricesAndCalculateProfits) ... ok
test_fetch_prices_and_calculate_profits (__main__.TestFetchPricesAndCalculateProfits) ... 

Fetching data from Coinbase API with URL: https://api.coinbase.com/v2/prices/BTC-USD/spot
Fetching data from Coinbase API with URL: https://api.coinbase.com/v2/prices/BTC-USD/spot
Fetching data from CryptoCompare API with URL: https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD
Fetching data from Kraken API with URL: https://api.kraken.com/0/public/Ticker?pair=BTCUSD
Fetching data from Bitstamp API with URL: https://www.bitstamp.net/api/v2/ticker/btcusd/
Fetching data from Coinbase API with URL: https://api.coinbase.com/v2/prices/BTC-USD/spot
Fetching data from Coinbase API with URL: https://api.coinbase.com/v2/prices/ETH-USD/spot
Fetching data from CryptoCompare API with URL: https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD
Fetching data from CryptoCompare API with URL: https://min-api.cryptocompare.com/data/price?fsym=ETH&tsyms=USD
Fetching data from Kraken API with URL: https://api.kraken.com/0/public/Ticker?pair=BTCUSD
Fetching data from Kraken API with U

ok
test_linked_list_display (__main__.TestLinkedList) ... 

Prices fetched: {'Coinbase': {'BTC': 66732.995, 'ETH': 3097.62}, 'CryptoCompare': {'BTC': 66727.88, 'ETH': 3097.65}, 'Kraken': {'BTC': 66695.6, 'ETH': 3095.98}, 'Bitstamp': {'BTC': 66707.0, 'ETH': 3095.9}}
Profits calculated: {'BTC': [{'buy_from': 'Coinbase', 'sell_to': 'CryptoCompare', 'profit': -5.115}, {'buy_from': 'Coinbase', 'sell_to': 'Kraken', 'profit': -37.395}, {'buy_from': 'Coinbase', 'sell_to': 'Bitstamp', 'profit': -25.995}, {'buy_from': 'CryptoCompare', 'sell_to': 'Coinbase', 'profit': 5.115}, {'buy_from': 'CryptoCompare', 'sell_to': 'Kraken', 'profit': -32.28}, {'buy_from': 'CryptoCompare', 'sell_to': 'Bitstamp', 'profit': -20.88}, {'buy_from': 'Kraken', 'sell_to': 'Coinbase', 'profit': 37.395}, {'buy_from': 'Kraken', 'sell_to': 'CryptoCompare', 'profit': 32.28}, {'buy_from': 'Kraken', 'sell_to': 'Bitstamp', 'profit': 11.4}, {'buy_from': 'Bitstamp', 'sell_to': 'Coinbase', 'profit': 25.995}, {'buy_from': 'Bitstamp', 'sell_to': 'CryptoCompare', 'profit': 20.88}, {'buy_from'

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/unittest/mock.py:1412: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  original = getattr(target, name, DEFAULT)


FAIL
test_node_creation (__main__.TestNode) ... ok

FAIL: test_linked_list_display (__main__.TestLinkedList)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/yf/pyrw7kxx273bsynvc3qrdcd00000gn/T/ipykernel_10509/4151069825.py", line 49, in test_linked_list_display
    mock_display.assert_has_calls(expected_calls)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/unittest/mock.py", line 966, in assert_has_calls
    raise AssertionError(
AssertionError: Calls not found.
Expected: [<MagicMock spec='HTML' id='4569724816'>,
 <MagicMock spec='HTML' id='4567731264'>,
 <MagicMock spec='HTML' id='4567737360'>]

----------------------------------------------------------------------
Ran 8 tests in 3.225s

FAILED (failures=1)
